In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Research the idea of computing a history of implied volatilities by always having the underlying price the same value
* **This requires plotly 4.0 or greater** *

Calculate the implied volatility of each days options as if the underlying futures contract were always some fixed value. For example, if the actual futures is 100, make that price 50, and then change all of the strike prices so that the strike prices are relative to the value 50.  

1. The 100 call would be a 50 dollar call.  
2. The 95 put would be a 45 dollar put.  
3. The 105 call would be a 55 dollar call


In [ ]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import pytz
import pathlib
from dateutil.relativedelta import *
import pandas_market_calendars as pmc

import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr
from dashapp import dashapp2 as dashapp

# importlib.reload(build_db)

### important global variables

In [ ]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


In [ ]:
def create_skew_per_date_df(df):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date
    
    df_ret = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret = df_ret.merge(df_temp,on='moneyness',how='outer')
        df_ret = df_ret.rename(columns={'vol_skew':str(settle_date)})
    df_ret = df_ret.sort_values('moneyness')
    df_ret.moneyness = df_ret.moneyness.round(4)
    return df_ret


In [ ]:
def graph_skew(df_iv_final,do_plot=False):
    '''
    Graph skew for ONLY ONE symbol.
    If df_iv_final contains more than one symbol, we will only graph the first symbol in the DataFrames    
    '''
    # get the first symbol (which should be the only symbol)
    contract = df_iv_final.symbol.unique()[0]
    # get just that symbol's data and only days that have sufficient skew data
    dft = df_iv_final[df_iv_final.symbol==contract]
    dft_count = dft[['settle_date','symbol']].groupby('settle_date',as_index=False).count()
    valid_settle_dates = dft_count[dft_count.symbol>2].settle_date.unique()
    dft = dft[dft.settle_date.isin(valid_settle_dates)]
    dfp = create_skew_per_date_df(dft)
    
    settle_dates = sorted([c for c in dfp.columns.values if c != 'moneyness'])
    splits = list(np.arange(5,len(settle_dates),5))
    settle_date_groups = np.split(np.array(settle_dates),splits)
    ret_figs = []
    for sdg in settle_date_groups:
        sdg_sorted = [str(c) for c in sorted(sdg)]
        cols = ['moneyness']+list(sdg_sorted)
        dfp_sub = dfp[cols]
        t = f'{contract} {sdg[0]} - {sdg[-1]}' 
        f = plotly_plot(dfp_sub,x_column='moneyness',plot_title=t,y_left_label='vol skew')
        ret_figs.append(f)
        if do_plot:
            iplot(f)
    return ret_figs

#### Do the same plots as above, but using a grid

In [ ]:

def graph_skew_subplot_quad(fig_group,rows=1,cols=2):
    '''
    Use subplots to output the results of the method graph_skew above
    '''
    f1 = make_subplots(rows=rows, cols=cols,  
        shared_yaxes=False,
        shared_xaxes=False,                       
        subplot_titles=[fig_group[i]['layout'].title.text for i in range(len(fig_group))],
        horizontal_spacing=0.05,
        vertical_spacing=0.11,                       
        print_grid=False)

    pl_width=450*cols 
    pl_height=400*rows
    title = 'Skew plots<br>'

    f1.update_layout(title=title,                                 
        font= Font(family="Open Sans, sans-serif"),
        showlegend=True,     
        hovermode='x',  
        autosize=True,       
        width=pl_width,       
        height=pl_height,
        plot_bgcolor='#EFECEA', 
        bargap=0.05,
        margin=Margin(
                      l=5,
                      r=5,
                      b=55,
                      t=50
        )
    )    

    for i in range(len(fig_group)):
        x = int(i/2) + 1
        y = i % 2 + 1
        f = fig_group[i]
        l = f.layout
        if y > 1:
            l.yaxis.title=''
        try:
            yaxis = f'yaxis{i+1}'
            xaxis = f'xaxis{i+1}'
            if i < 10:
                yaxis = yaxis.replace('1','') 
                xaxis = xaxis.replace('1','') 
            gname = f'{x,y}'#rfs[i]['layout'].title
            for d in f.data:
                data_y = f'y{i+1}'.replace('1','') 
                d['yaxis']=data_y
                d['legendgroup'] =  gname
                d['name'] = f"{d.name}"
                f1.add_trace(d,x,y)
                f1.update_xaxes(patch=l.xaxis,row=x,col=y)
                f1.update_yaxes(patch=l.yaxis,row=x,col=y)                
        except Exception as e:
            print(f'graph_skew_subplots ERRORS: {str(e)}')
    return f1


def graph_skew_subplots(df,rows=2,cols=2):
    fig_list = graph_skew(df)
    n = rows*cols   
    # using list comprehension 
    fig_groups = [fig_list[i*n:(i + 1)*n] for i in range((len(fig_list) + n - 1) // n )]  
    for fig_group in fig_groups:
        iplot(graph_skew_subplot_quad(fig_group,rows=rows,cols=cols))


### Graph skew changes historically, per day

In [ ]:
SYMBOL_TO_RESEARCH = 'CL'
all_contracts = pga.get_sql(f"select symbol from {opttab} where substring(symbol,1,2)='{SYMBOL_TO_RESEARCH}'").symbol.unique()
all_contracts 

In [ ]:
uk_holidays = open('expiration_data/uk_holidays.csv').readlines()
# uksplit = [','.join([t.strip('\n') for t in l.split(',')]) for l in  uk_holidays]
uksplit = [','.join(l.split(',')) for l in  uk_holidays]
fio = io.StringIO()
fio.writelines(uksplit)
fio.seek(0)
df_ukh = pd.read_csv(fio)
def ukh_to_yyyymmdd(month_day,year):
    md = month_day.strip()
    d = datetime.datetime.strptime(f'{md} {year}', '%B %d %Y')
    return d
#     return d.strftime("%Y-%m-%d")

year_cols = [c for c in df_ukh.columns.values if '20' in str(c)]
for c in year_cols:
    df_ukh[c] = df_ukh[c].apply(lambda s:ukh_to_yyyymmdd(s,c))
uk_holidays = sorted(df_ukh[year_cols].values.reshape(-1))
uk_holidays

In [ ]:
MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}

from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
# bday_uk = pd.offsets.CustomBusinessDay(calendar=pmc.exchange_calendar_ice.ICEExchangeCalendar().regular_holidays)
bday_uk = pd.offsets.CustomBusinessDay(holidays=uk_holidays)
TIMEZONE = 'US/Eastern'


In [ ]:

def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)




def get_ES_expiry(symbol):
    '''
    3rd friday of month of symbol
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_E6_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_CL_expiry(symbol):
    '''
    Trading terminates 7 business days before the 26th calendar of the month prior to the contract month.
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_NG_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

def get_CB_expiry(symbol):
    '''
    This is the spec for the CME Brent, but it matches ICE.
    Trading terminates the 4th last London business day of 
    the month, 2 months prior to the contract month 
    except for the February contract month which 
    terminates the 5th last London business day of the 
    month, 2 months prior to the contract month.  
    '''
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month - 1
    if month<1:
        month = 12 + month
        year = year - 1
    days_to_subtract = 4
    if monthcode_yy[0] =='G':
        days_to_subtract = 5
    elif monthcode_yy[0] == 'F':
        days_to_subtract = 3
#     elif monthcode_yy == 'N22':
#         days_to_subtract = 7
    return datetime.datetime(year,month,1,0,0) - days_to_subtract * bday_uk

DICT_PRODUCT = {
    'E6':get_E6_expiry,
    'ES':get_ES_expiry,
    'CL':get_CL_expiry,
    'NG':get_NG_expiry,
    'CB':get_CB_expiry,
}

    
def get_expiry(symbol):
    product = symbol[:2]
    if product not in DICT_PRODUCT:
        return None
    f = DICT_PRODUCT[product]
    return f(symbol)


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365


In [ ]:
def get_postgres_data(contract,pga):
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    usql = f"select * from {futtab} where symbol='{contract}';"
    dfu = pga.get_sql(usql)
    # Merge options and futures data
    df = dfo.merge(dfu,how='inner',on=['symbol','settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    calculated_expiry = get_expiry(contract)    
    if calculated_expiry is not None:
        expiry_yyyymmdd = yyyymmdd_from_dt(calculated_expiry)
        if expiry_yyyymmdd> df_expiry_dates.iloc[0].settle_date:
            df_expiry_dates.loc[0,'settle_date'] = expiry_yyyymmdd
    return df,df_expiry_dates

In [ ]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df9 = df2.copy()
    df9['expiry'] = df_expiry_dates.iloc[0].settle_date
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



In [ ]:
def get_even_moneyness_strikes(df10):
    # define amounts around the money which will help create strikes to add
    moneyness = np.arange(.7,1.4,.05).round(6)
    # define columns on which to execute groupby
#     gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(4)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')
    df12.moneyness = df12.moneyness.round(4)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
    return df12



In [ ]:
def get_contracts(commod):
    all_contracts = pga.get_sql(f"select symbol from {opttab} where substring(symbol,1,2)='{commod}'").symbol.unique()
    return all_contracts


def pseudo_vol(df,df_expiry,fatm=50,rows=100):
    '''
    get implied vol as if every underlying was the price specified in the arg fatm
    :param df:  DataFrame with columns from get_postgres_data of:
    'symbol','strike','pc','settle_date','close_x','close_y','contract_num'
    :param df_expiry: a one row DataFrame with the expiration date of the options on 
       the symbol whose options data is in df
    :param fatm: the futures atm price to use as the underlying futures for all
      implied vol calculations
    
    '''
    # get desired columns
    df_cl2 = df[['symbol','strike','pc','settle_date','close_x','close_y','contract_num']]
    # find the atm_op: the at the money option (closest to the money)
    df_cl2['atm_op'] = [abs(v) for v in (df_cl2.strike - df_cl2.close_y)]
    df_cl2_atm_op = df_cl2[['settle_date','atm_op']].groupby('settle_date',as_index=False).min()
    df_cl2 = df_cl2_atm_op.merge(df_cl2,on=['settle_date','atm_op'],how='inner')
    df_cl2.index = list(range(len(df_cl2)))
    # get either the put or the call with the same atm_op value and lowest close_x value
    #   close_x is the options price
    df_cl2_put_or_call = df_cl2[['settle_date','close_x']].groupby('settle_date',as_index=False).min()
    df_cl2 = df_cl2_put_or_call.merge(df_cl2,on=['settle_date','close_x'],how='inner')

    # create a new strike
    df_cl2.strike = fatm + df_cl2.strike - df_cl2.close_y
    df_cl2['fprice'] = df_cl2.close_y
    df_cl2.close_y = fatm
    df_cl2 = df_cl2.iloc[-rows:]
    df_cl_fut = df_cl2[['settle_date','fprice']].drop_duplicates()
    df_cl_implieds = get_implieds(df_cl2,df_expiry)
    df_cl_implieds2 = df_cl_implieds[df_cl_implieds.dte>0]
    df_cl_implieds2 = df_cl_implieds2.merge(df_cl_fut,on='settle_date')
    return df_cl_implieds2[['settle_date','iv','fprice']]

#     df_cl_implieds2['atm_op'] = df_cl_implieds2.apply(
#         lambda r:abs(r.strike - r.close_y),axis=1)
#     df_cl_implieds3 = df_cl_implieds2[['settle_date','atm_op']].groupby('settle_date',as_index=False).min()
#     df_cl_implieds4 = df_cl_implieds3.merge(
#         df_cl_implieds2[['settle_date','atm_op','iv']],on=['settle_date','atm_op'],how='inner')
#     df_cl_implieds4 = df_cl_implieds4[['settle_date','iv']]
#     df_cl_implieds4 = df_cl_implieds4.merge(df_cl_fut,on='settle_date')
#     return df_cl_implieds4

In [ ]:
df_test = pseudo_vol(dict_contract_df['CLM19'][0],dict_contract_df['CLM19'][1])
display.display(df_test)

In [ ]:
def ordered_symbols(sym_list):
    ordered_keys = [
        s[:2]+s[-1]+s[2:4] 
        for s in sorted([v[0:2]+v[-2:]+v[2] for v in sym_list])
    ]
    return ordered_keys
    

In [ ]:
def loop_pseudo_vol(dict_contract_df,pga,
                   fatm=50,rows=100):
#     ordered_keys = [
#         s[:2]+s[-1]+s[2:4] 
#         for s in sorted([v[0:2]+v[-2:]+v[2] for v in dict_contract_df.keys()])
#     ]
    ordered_keys = ordered_symbols(list(dict_contract_df.keys()))
    
    dict_df_implied = {}

    for symbol in tqdm_notebook(ordered_keys):
        if dict_contract_df[symbol] is None:
            df,df_expiry = get_postgres_data(symbol,pga)
        else:
            df,df_expiry = dict_contract_df[symbol]
        try:
            df_implied = pseudo_vol(df,df_expiry,fatm=fatm,rows=rows)
            dict_df_implied[symbol] = df_implied
        except Exception as e:
            print(str(e))
            continue
    return dict_df_implied

In [ ]:
dict_contract_df = {c:None for c in get_contracts('CL')}

In [ ]:
for symbol in tqdm_notebook(dict_contract_df.keys()):
    dict_contract_df[symbol] = get_postgres_data(symbol,pga)

In [ ]:
# dict_small_cdf = {c:dict_contract_df[c] for c in dict_contract_df.keys() if ((c[2] >'N') and (int(c[-2:]) >= 20))}
# dict_df_implied = loop_pseudo_vol(dict_small_cdf,pga)
dict_df_implied = loop_pseudo_vol(dict_contract_df,pga)


In [ ]:
dict_df_implied['CLM19']

In [ ]:
year = 2020
for symbol in [k for k in dict_df_implied.keys() if int(k[-2:])==year-2000]:
    df_to_graph = dict_df_implied[symbol][:-1]
    fig = dashapp.plotly_plot(
        df_in=df_to_graph,x_column='settle_date',yaxis2_cols=['fprice'],
        y_left_label='Implied Volatility',y_right_label='Futures Close',
        plot_title=f"{symbol} Pseudo-Volatility vs Price"
    )
    iplot(fig)


## END

In [ ]:
df_cl_all = None
df_cl_all_expiry = None
for symbol in tqdm_notebook(dict_contract_df.keys()):
    df_temp = dict_contract_df[symbol][0]
    df_temp_expiry = dict_contract_df[symbol][1]
    if df_cl_all is None:
        df_cl_all = df_temp.copy()
        df_cl_all_expiry = df_temp_expiry.copy()
    else:
        df_cl_all = df_cl_all.append(df_temp,ignore_index=True)
        df_cl_all_expiry = df_cl_all_expiry.append(df_temp_expiry,ignore_index=True)
    df_cl_all.index = list(range(len(df_cl_all)))
    df_cl_all_expiry.index = list(range(len(df_cl_all_expiry)))
    

In [ ]:
h = pathlib.Path.home()
fname = f"{h}/downloads/df_cl_all.pickle"
df_cl_all.to_pickle(fname)
